# Model C (log spectrogram input)
## With: multi-hot labelling and binary cross-entropy loss.

In [1]:
import sys
import os
import tensorflow as tf
from tensorflow.python.client import device_lib
from keras import backend as K
from keras.layers import Activation, Add, Conv2D, Dense, Dropout, Flatten, Input, LeakyReLU
from keras.losses import kullback_leibler_divergence
from keras.metrics import binary_accuracy, categorical_accuracy
from custom_metric import rounded_all_or_nothing_acc as RAON_accuracy
from keras.models import Model
from generator import AudioGenerator, kltls, multilabelled_labels_to_ys, multilabelled_ys_to_labels, onehot_superclass_labels_to_ys, onehot_superclass_ys_to_labels, MULTI_LABEL, ONE_HOT, TIME_SEQUENCE, LOG_SPECTROGRAM, LINEAR_SPECTROGRAM
import pickle
import numpy as np
import datetime
import math
from keras.callbacks import TensorBoard
from time_callback import Time_Callback

# Allows me to import my modules
sys.path.append('./modules')
from audio_utils import *

Using TensorFlow backend.


Attempting to read settings file...
	Read successfully!


In [2]:
# This makes sure that Tensorflow has access to my GPU, as well as my CPU.
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11284613977366837717, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 577778483
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7238689346923368571
 physical_device_desc: "device: 0, name: GeForce GTX 650, pci bus id: 0000:01:00.0, compute capability: 3.0"]

In [3]:
# Tells Tensorflow to use the GPU
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)

config = tf.ConfigProto(allow_soft_placement=True,
                        device_count = {'CPU' : 1,
                                        'GPU' : 1},
                        log_device_placement = True,
                        gpu_options=gpu_options
                       )

#Loss flags
BINARY_CROSSENTROPY = 0
KL_DIVERGENCE = 1
LOSSES = {BINARY_CROSSENTROPY: binary_crossentropy, KL_DIVERGENCE: kullback_leibler_divergence}

# Model config
problem_type = MULTI_LABEL
input_type = LOG_SPECTROGRAM
loss = BINARY_CROSSENTROPY
optimizer = "adam"

# Name of model save and log
problem_type_str = "OneHot" if problem_type == ONE_HOT else "MultiHot"
input_type_str = {TIME_SEQUENCE: "(1D)", LINEAR_SPECTROGRAM: "(linear 2D)", LOG_SPECTROGRAM: "(log 2D)"}[input_type]  
loss_str = "BCE" if loss == BINARY_CROSSENTROPY else "KLD"
optimizer_str = optimizer.upper()
date_time_str = str(datetime.datetime.now().strftime('%d-%m-%Y_%H-%M-%S'))
model_name = "ModelC-{}-{}-{}-{}_{}".format(input_type_str, problem_type_str, loss_str, optimizer_str, date_time_str)

In [4]:
# Data generators
batch_size = 50
generators = {"training": None, "validation": None, "test": None}
N = {"training": 0, "validation": 0, "test": 0}
for data_type in generators.keys():
    sample_metadata = get_file_classes(data_type)
    N[data_type] = len(sample_metadata)
    filenames = [sm["filepath"] for sm in sample_metadata]
    labels = [sm["labels"] for sm in sample_metadata]
    generators[data_type] = AudioGenerator(filenames, labels, data_type, batch_size, shuffle=True, problem_type=problem_type, input_type=input_type)

In [5]:
batch_x_shape = None
batch_y_shape = None 
for name, gen in generators.items():
    print("Gen", name)
    batch_0 = gen.__getitem__(0)
    print("In shape:", batch_0[0].shape, "\nOut shape:", batch_0[1].shape)
    batch_x_shape = batch_0[0].shape
    batch_y_shape = batch_0[1].shape

Gen training
In shape: (50, 129, 53, 1) 
Out shape: (50, 10)
Gen validation
In shape: (50, 129, 53, 1) 
Out shape: (50, 10)
Gen test
In shape: (50, 129, 53, 1) 
Out shape: (50, 10)


In [6]:
# Test whether generator arguments are picklable (whether they can be multiprocessed)
use_multiprocessing = True
for gen in generators:
    try:
        pickle.dumps(gen)
    except:
        print(sys.exc_info())
        use_multiprocessing = False
        break
print("Picklable:", use_multiprocessing)

Picklable: True


## Rationale

### Model structure

2 "2DConvAct" convolution layers which reduce the size of the sample space while increasing the size of the convolution/feature space. Causal convolutions don't exist for 2D (yet) so fully connected 2D convolution will suffice.

Creates feature space of 32, while downscaling the sample space to 33x14 (462). Compared to 129x53 (6837), total tensor sizes: 341850 -> 739200 (increase in data).

After convoluton layers, LeakyReLU was used for activation because ReLU has been shown to perform well and LeakyReLU takes negatives into account slightly which appear in the data. For this reason He-normal was used to initialise the convolution kernals as this performs well with ReLU.

Then 3 lots of "DilatedDropoutSkipModule" which retain the size of the sample space while extracting more features. With skip connections preserving earlier features. Finished with dropout layers to aid in generalisation during training.

Flattened and passed to a fully-connected (dense layer) which reshapes the network into the output shape.

Softmax activation layer for one-hot classification.

Techniques from lit review:
- LeNet: Convolutions with `stride > 1` to downscale sample space.
- Using 1x1 convolution layers to downscale feature space, instead of pooling layers. 
- Leaky ReLU & He kernal inits.
- Dropout for generalisation during training.
- ResNet for skip connections.

In [7]:
# Reusable series of layers
def DilatedDropoutSkipModule(og_model, drop_rate):
    model = Conv2D(filters=1, kernel_size=1, padding="same", dilation_rate=1, kernel_initializer='he_normal')(og_model)
    model = Conv2D(filters=8, kernel_size=3, padding="same", dilation_rate=2, kernel_initializer='he_normal')(model)
    model = Conv2D(filters=32, kernel_size=3, padding="same", dilation_rate=2, kernel_initializer='he_normal')(model)
    model = LeakyReLU(0.3)(model)
    model = Add()([og_model, model])
    return Dropout(rate=drop_rate)(model)

# The dropout rates and amount of modules.
NMODULES = 3
DROP_RATES = [0.15, 0.2, 0.25]

final_activation = {ONE_HOT: "softmax", MULTI_LABEL: "sigmoid"}[problem_type]

# Structure
data = Input(shape=(129, 53, 1))
cnn = Conv2D(filters=16, kernel_size=5, strides=2, padding="same", dilation_rate=1, kernel_initializer='he_normal')(data)
cnn = LeakyReLU(0.3)(cnn)
cnn = Conv2D(filters=32, kernel_size=3, strides=2, padding="same", dilation_rate=1, kernel_initializer='he_normal')(cnn)
cnn = LeakyReLU(0.3)(cnn)
cnn = Dropout(rate=0.1)(cnn)
for i in range(NMODULES):
    cnn = DilatedDropoutSkipModule(cnn, DROP_RATES[i])
cnn = Flatten()(cnn)
cnn = Dense(batch_y_shape[1], kernel_initializer='he_normal')(cnn)
cnn = Activation(final_activation)(cnn)
model = Model(inputs=data, outputs=cnn)

for layer in model.layers:
    print(layer.name, layer.output_shape)

# The label specific metric, dependant on the problem type from custom settings.
problem_metric = {ONE_HOT: categorical_accuracy, MULTI_LABEL: binary_accuracy}[problem_type]
    
# Compile with custom settings defined earlier
model.compile(optimizer=optimizer, loss=LOSSES[loss], metrics=[problem_metric, RAON_accuracy, LOSSES[(loss + 1)%2]])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
input_1 (None, 129, 53, 1)
conv2d_1 (None, 65, 27, 16)
leaky_re_lu_1 (None, 65, 27, 16)
conv2d_2 (None, 33, 14, 32)
leaky_re_lu_2 (None, 33, 14, 32)
dropout_1 (None, 33, 14, 32)
conv2d_3 (None, 33, 14, 1)
conv2d_4 (None, 33, 14, 8)
conv2d_5 (None, 33, 14, 32)
leaky_re_lu_3 (None, 33, 14, 32)
add_1 (None, 33, 14, 32)
dropout_2 (None, 33, 14, 32)
conv2d_6 (None, 33, 14, 1)
conv2d_7 (None, 33, 14, 8)
conv2d_8 (None, 33, 14, 32)
leaky_re_lu_4 (None, 33, 14, 32)
add_2 (None, 33, 14, 32)
dropout_3 (None, 33, 14, 32)
conv2d_9 (None, 33, 14, 1)
conv2d_10 (None, 33, 14, 8)
conv2d_11 (None, 33, 14, 32)
leaky_re_lu_5 (None, 33, 14, 32)
add_3 (None, 33, 14, 32)
dropout_4 (None, 33, 14, 32)
flatten_1 (None, 14784)
dense_1 (None, 10)
activation_1 (None, 10)


In [8]:
# Training logs
log_dir = "logs/{}".format(model_name)
# Tensorboard log
tb_log = TensorBoard(log_dir=log_dir)
# Custom time log
time_log = Time_Callback(log_dir=log_dir)

# Train model
epochs = 10
dataset_perc = 1
training_history = model.fit_generator(
                generator = generators["training"],
                steps_per_epoch = int(N["training"]*dataset_perc) // batch_size,
                validation_data = generators["validation"],
                validation_steps = int(N["validation"]*dataset_perc) // batch_size,
                epochs = epochs,
                callbacks = [tb_log, time_log],
                use_multiprocessing = use_multiprocessing,
                workers = 4
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1587/1587 [==============================] - 1445s 910ms/step - loss: 0.5061 - acc: 0.8016 - rounded_all_or_nothing_acc: 0.0267 - kullback_leibler_divergence: 3.1963 - val_loss: 0.4323 - val_acc: 0.8135 - val_rounded_all_or_nothing_acc: 0.0393 - val_kullback_leibler_divergence: 3.3157
Epoch 2/10
1587/1587 [==============================] - 1334s 840ms/step - loss: 0.3375 - acc: 0.8605 - rounded_all_or_nothing_acc: 0.1895 - kullback_leibler_divergence: 2.0818 - val_loss: 0.2840 - val_acc: 0.8840 - val_rounded_all_or_nothing_acc: 0.2602 - val_kullback_leibler_divergence: 1.8650
Epoch 3/10
1587/1587 [==============================] - 1189s 749ms/step - loss: 0.2852 - acc: 0.8851 - rounded_all_or_nothing_acc: 0.2827 - kullback_leibler_divergence: 1.7511 - val_loss: 0.2515 - val_acc: 0.9016 - val_rounded_all_or_nothing_acc: 0.3370 - val_kullback_leibler_divergence: 1.5491
Epoch 4/10
1587/1587 [==============================] - 2542s

### Evaluation of model

After 10 epochs of training and validation with the training and validation data sets, a seperate evaluation run calculated the final accuracy of the model using the test data set.

In [9]:
# Evaluate model
model.evaluate_generator(
    generators["test"],
    int(N["test"]*dataset_perc) // batch_size,
    use_multiprocessing = use_multiprocessing,
    workers = 4
)

[0.19478438286618752,
 0.9251249974424188,
 0.4533333223821087,
 1.2430265752147094]

In [10]:
# Save model and weights to `/models` directory
save_dir = os.path.join(os.getcwd(), "models")
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
model_json = model.to_json()
with open(os.path.join(save_dir, "{}.json".format(model_name)), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(save_dir, "{}.h5".format(model_name)))
print("Saved model to disk")

Saved model to disk


### Some examples of predictions using test data set

In [11]:
generators["test"].on_epoch_end()
batch0_test = generators["test"].__getitem__(0)

In [12]:
preds, trues = [], []
for i in range(10):
    x, y = batch0_test[0][i], batch0_test[1][i]
    pred_y = np.reshape(model.predict(x.reshape(1, 129, 53, 1)), batch_y_shape[1])
    print("Sample", i)
    print("Actual:\n\t{},\n\t{}\nPrediction:\n\t{},\n\t{}\n".format(y, onehot_superclass_ys_to_labels(y), [round(p_y, 3) for p_y in pred_y.tolist()], onehot_superclass_ys_to_labels([int(round(p_y)) for p_y in pred_y.tolist()])))
    preds.append(pred_y)
    trues.append(y)

preds_tensor = K.variable(np.array(preds))
trues_tensor = K.variable(np.array(trues))
problem_acc = K.eval(K.mean(problem_metric(trues_tensor, preds_tensor)))
print("{} accuracy (avg): {}".format("Categorical" if problem_type is ONE_HOT else "Binary", problem_acc))
print("RAON accuracy: ", K.eval(RAON_accuracy(trues_tensor, preds_tensor)))

Sample 0
Actual:
	[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.],
	{'hit_label': ['stick'], 'kit_label': ['hi_hat'], 'tech_label': ['open']}
Prediction:
	[0.133, 0.0, 0.0, 1.0, 0.008, 0.001, 0.001, 0.017, 0.01, 0.013],
	{'hit_label': ['stick'], 'kit_label': ['hi_hat'], 'tech_label': ['open']}

Sample 1
Actual:
	[1. 0. 1. 0. 0. 0. 0. 0. 1. 0.],
	{'hit_label': ['beater'], 'kit_label': ['bass_drum'], 'tech_label': ['normal']}
Prediction:
	[0.298, 0.011, 0.511, 0.002, 0.08, 0.061, 0.002, 0.289, 0.402, 0.004],
	{'hit_label': ['stick'], 'kit_label': ['hi_hat'], 'tech_label': ['normal']}

Sample 2
Actual:
	[0. 0. 0. 1. 1. 0. 0. 0. 0. 0.],
	{'hit_label': ['stick'], 'kit_label': ['hi_hat'], 'tech_label': ['open']}
Prediction:
	[0.219, 0.05, 0.02, 0.918, 0.388, 0.353, 0.03, 0.016, 0.216, 0.001],
	{'hit_label': ['stick'], 'kit_label': ['hi_hat'], 'tech_label': ['open']}

Sample 3
Actual:
	[1. 0. 0. 0. 0. 1. 0. 1. 0. 0.],
	{'hit_label': ['beater'], 'kit_label': ['bass_drum'], 'tech_label': ['normal']}
Prediction